**Group 3**: Loïk Vuilleumier (283675), Marc Favier (319571), Michael Biselx (283812), Samuel Bumann (284704)

---
# Basics of Mobile Robotics Miniproject
---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basics-of-Mobile-Robotics-Miniproject" data-toc-modified-id="Basics-of-Mobile-Robotics-Miniproject-1">Basics of Mobile Robotics Miniproject</a></span></li><li><span><a href="#Introduction:" data-toc-modified-id="Introduction:-2">Introduction:</a></span><ul class="toc-item"><li><span><a href="#Environment" data-toc-modified-id="Environment-2.1">Environment</a></span></li></ul></li><li><span><a href="#Image-Processing" data-toc-modified-id="Image-Processing-3">Image Processing</a></span><ul class="toc-item"><li><span><a href="#Detecting-corner-of-the-robot's-environment" data-toc-modified-id="Detecting-corner-of-the-robot's-environment-3.1">Detecting corner of the robot's environment</a></span></li><li><span><a href="#The-image-processing-can-be-divided-into-four-steps-:" data-toc-modified-id="The-image-processing-can-be-divided-into-four-steps-:-3.2">The image processing can be divided into four steps :</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Obtain-an-image-of-the-region-of-interest-:-the-map-of-the-environment." data-toc-modified-id="Step-1:-Obtain-an-image-of-the-region-of-interest-:-the-map-of-the-environment.-3.2.1">Step 1: Obtain an image of the region of interest : the map of the environment.</a></span></li><li><span><a href="#Step-2:-Find-the-position-of-the-robot." data-toc-modified-id="Step-2:-Find-the-position-of-the-robot.-3.2.2">Step 2: Find the position of the robot.</a></span></li><li><span><a href="#Step-3:-Find-the-goal." data-toc-modified-id="Step-3:-Find-the-goal.-3.2.3">Step 3: Find the goal.</a></span></li><li><span><a href="#Step-4:-Get-a-map-for-the-global-path-planning." data-toc-modified-id="Step-4:-Get-a-map-for-the-global-path-planning.-3.2.4">Step 4: Get a map for the global path planning.</a></span></li></ul></li><li><span><a href="#Testing-the-Module" data-toc-modified-id="Testing-the-Module-3.3">Testing the Module</a></span></li></ul></li><li><span><a href="#Global-Path-Planning" data-toc-modified-id="Global-Path-Planning-4">Global Path Planning</a></span><ul class="toc-item"><li><span><a href="#Different-Approaches" data-toc-modified-id="Different-Approaches-4.1">Different Approaches</a></span><ul class="toc-item"><li><span><a href="#A*-on-grid-array" data-toc-modified-id="A*-on-grid-array-4.1.1">A* on grid array</a></span></li><li><span><a href="#A*-algorithm-theory" data-toc-modified-id="A*-algorithm-theory-4.1.2">A* algorithm theory</a></span></li></ul></li><li><span><a href="#Testing-the-A*-Implementation" data-toc-modified-id="Testing-the-A*-Implementation-4.2">Testing the A* Implementation</a></span></li></ul></li><li><span><a href="#Localistion-and-Control" data-toc-modified-id="Localistion-and-Control-5">Localistion and Control</a></span><ul class="toc-item"><li><span><a href="#Theory" data-toc-modified-id="Theory-5.1">Theory</a></span><ul class="toc-item"><li><span><a href="#Estimator" data-toc-modified-id="Estimator-5.1.1">Estimator</a></span></li><li><span><a href="#Control-Signal" data-toc-modified-id="Control-Signal-5.1.2">Control Signal</a></span></li><li><span><a href="#State" data-toc-modified-id="State-5.1.3">State</a></span></li><li><span><a href="#Motion-Model" data-toc-modified-id="Motion-Model-5.1.4">Motion Model</a></span></li><li><span><a href="#Sensor-interpretation" data-toc-modified-id="Sensor-interpretation-5.1.5">Sensor interpretation</a></span></li></ul></li><li><span><a href="#Setup" data-toc-modified-id="Setup-5.2">Setup</a></span></li></ul></li><li><span><a href="#Local-Navigation" data-toc-modified-id="Local-Navigation-6">Local Navigation</a></span></li><li><span><a href="#Final-Program" data-toc-modified-id="Final-Program-7">Final Program</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-8">Results</a></span><ul class="toc-item"><li><span><a href="#Normal-Run" data-toc-modified-id="Normal-Run-8.1">Normal Run</a></span></li><li><span><a href="#No-Vision" data-toc-modified-id="No-Vision-8.2">No Vision</a></span></li><li><span><a href="#Kidnapping" data-toc-modified-id="Kidnapping-8.3">Kidnapping</a></span></li></ul></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-9">Conclusion</a></span><ul class="toc-item"><li><span><a href="#Image-Processing" data-toc-modified-id="Image-Processing-9.1">Image Processing</a></span></li><li><span><a href="#Global-Path-Planning" data-toc-modified-id="Global-Path-Planning-9.2">Global Path Planning</a></span></li><li><span><a href="#Localisation-and-Control" data-toc-modified-id="Localisation-and-Control-9.3">Localisation and Control</a></span></li><li><span><a href="#Local-Navigation" data-toc-modified-id="Local-Navigation-9.4">Local Navigation</a></span></li><li><span><a href="#Overall" data-toc-modified-id="Overall-9.5">Overall</a></span></li></ul></li></ul></div>

# Introduction:
The objective of this project is to create a program in Python/Aseba that:
- Uses the visual feedback of a webcam to detect the starting position of a robot, its goal position and 2D obstacles in a given environment with some known specifications described in the following section Environment.
- Finds the optimal path from the start position of the robot to its goal between the globals obstacles. It uses A* algorithm.
- Controls the robot to follow the optimal path, by successively applying filters to estimate its position. The entries are the absolute position of the robot given by the camera, the odometry and proximity and black and white sensors underneath the robot.
- Navigates around 3D local obstacles using local avoidance.

## Environment
The environment consists of a white table with a thin grid pattern (each square is 200mm x 200mm). The table has blue disks in all corners. The Thymio robot is located within the rectangle spanned by the blue circles, along with a green disk that represents the goal position and black shapes which represent the obstacles. Two red disks of different diameters are located on top of the Thymio. The global obstacles are in 2D because 3D obstacles with an non vertical view (and even whith such a view) changes a lot in size. For local avoidance we use white 3D obstacles, that are not seen by the vision module.

> <img src="./imgs/environment_titel.jpg"  style="width: 400px;"/>

> *Figure 1: This figure shows the environment: the table with its grid pattern and the blue circles in the corners, the green goal circle, the black obstacle shapes and the Thymio robot with two red circles.*


# Image Processing
## Detecting corner of the robot's environment
We want to get rid off all the random components introduced by taking a picture at a skewed angle. This includes translations, rotations, scaling and most importantly perspective transformations. Therefore we want to search for four points, which we willingly place at known distances and angles from each other, to standardize our image. 

> <img src="./imgs/original_map.png" alt="original_map" style="width: 400px;"/>

> *Figure 2: Here is an example of the image we are working with.*

## The image processing can be divided into four steps :

### Step 1: Obtain an image of the region of interest : the map of the environment.
The best is to detect the four extreme angles of the rectangle environment. It's increases the precision of the transformation by decreasing the relative error on their distance to each other. There are several ways to find these corners : 

- detect the lines of the rectangle (hough transform or edge detection) but there will be too many lines with obstacles. It could have been possible to detect the edges of the outer rectangle defining the map, fill the obtained rectangle (meaning put all the pixels inside its closed borders to one), next erode the image to delete all exterior components to the rectangle, our region of interest. Then we just have to dilate to have only the rectangle with its original size on a black background, which is very easy to detect. But our background has too many long lines throughout the whole image (for example legs of the table) then filling closed borders would have lead to filling most of the image. To summarize, this is not a good method because our region of interest does not produce the biggest closed border, it's included in other closed borders.
- convolve the image with a corner (two small lines 90 degrees to each others) has the advantage of being quite scale invariant but depends on rotation. The results were very bad as the image often presents rotations. Furthermore with the image with grid pattern we can't distinguish all the corners.
- convolve with a disk has the advantage of being rotation invariant, which is better than scale invariant. So we introduce specific patterns at the corners to indirectly detect them. Its better to have disk with specific color, but even then convolution was not good enough (with perspective, disk become ellipsoids and do not fit the pattern anymore).
- use the HSV (Hue Saturation Value) representation of the image to easily detect colors. We pass from RGB representation where it's hard to detect specific colors to the HSV where saturated colors (meaning very red, very blue or very green) are easy to detect on a neutral backgound. Therefore we just need colors, without specific shape. We kept the disks because they are more robust to bad detection : when some pixels are not detected they mostly stay as one connected region of pixels with the same centroid.
We chose the last option, the most efficient.

Therefore four blue disks are placed next to the four corners, we filter those blue colors of the image and we obtain a mask which is one for each of those blue pixels and zero four the others. We apply an erosion followed by a dilation of the same size to remove all the possibles noisy isolated pixels which could have been considered as blue. In fact if our selection of pixels is too restrictive we might not detect the disks and if it's too large we might detect unexpected regions as blue. Next, we apply a labeling of connected components. Meaning that we associate a unique value to each region of pixels which are linked to each others. We ideally want five labels : one for the background and one for each of the four corners. But even with erosion, there might be some unexpected one values in the mask. Therefore we sort all the labels with respect to their number of pixels and we take the four biggest. After that we compute the centroides of those four regions. We put also all the blue pixels to white in order to avoid to detect them again in the others functions. Finally we apply the opencv function "cv2.getPerspectiveTransform" to fit the detected corners to defined values : the corners of an image of size (1200, 800). We obtain the following image :

> <img src="./imgs/cropped_map.png" alt="cropped_map" style="width: 400px;"/>

> *Figure 3: This figure shows a map of the robot's environment after transformation.*

### Step 2: Find the position of the robot.
We apply a similar operation to find the robot. We go through the HSV representation of the obtained map and filter the red components. We sort the labels by the number of elements and from here the function is different. We take the centroid of the biggest label, in terms of number of pixels, as the robot position in (x,y) coordinate and we use the centroid of the second biggest label, again in terms of number of pixels, to compute the robot's angle.

Finally, a heuristic correction on the position is applied, to compensate for the effect of the height of the robot and the camera's field of view.

### Step 3: Find the goal.
We again go through the HSV representation, filter green components and compute the centroid of the goal.

### Step 4: Get a map for the global path planning.
Here we go from RGB to grayscale representation of the environment. We apply an erosion and dilation to get rid off the line used for local positioning. We binarize the image using a fixed value. Theorically Otsu method is supposed to work better as it maximizes interclass variance end minimising intraclass variance. But for our case, a fixed threshold was enough and even a bit better. We use this binary image to find the global path.

> <img src="./imgs/map_bin.png" alt="map_bin" style="width: 400px;"/>

> *Figure 4: The binary map with dilated obstacles.*


The steps 1 and 2 are repeated while the robot is moving. We don't really need to do each time the step 1 which recomputes the map but this ensures that even if the camera moves a bit the position of the robot computed in step 2 is still valid. It's a security step. 



In [ ]:
#!pip install scikit-image  # used inside the Vision module

The following cell imports the necessary libraries and modules. The image processing module can be found under [Vision](src/Vision.py).

In [ ]:
import os
import sys
import cv2
import numpy as np

# Adding the src folder in the current directory as it contains the Vision module
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))
import Vision as vis

In [ ]:
def camera_thread(cap, camera_pos) : 
    """ do the camera position acquisition cycle """
    
    x = np.array([])
    while not x.size :
        success, img = cap.read()                                  # read the image from the fixed camera
        if success :
            success2, true_map     = vis.get_clean_map(img, [1200, 800])     # get the clean in good resolution
            if success2 :
                x            = vis.robot_detection(true_map)           # find the robot
            else:
                break
        time.sleep(.1)
    
    camera_pos.x = x.copy()

## Testing the Module
The following cell allows the testing of the setup, independently of other modules.

In [ ]:
# # get img from the webcam
# cap_size  = (1200, 1000)
# map_shape = [1200,  800]
# cap       = cv2.VideoCapture(2)                       # put 1 if an external webcam is used
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_size[0])        # width
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT,cap_size[1])        # height
# cap.set(cv2.CAP_PROP_AUTO_EXPOSURE,1)                 # set auto exposure

# robot_pos = np.array([])

# while True :
#     success, img = cap.read()
#     if success :
#         success2, true_map = vis.get_clean_map(img, map_shape)                  # get a clean map
#         if success2 :
#             robot_pos = vis.robot_detection(true_map)                           # find the robot, annotate on map
#             goal_pos  = vis.goal_detection(true_map)                            # find the goal, annotate on map
#             if robot_pos.size :
#                 obs_map = vis.get_global_search_map(true_map, robot_pos)        # find obstacles
#                 cv2.imshow("Obstacles", obs_map)

#             true_map = vis.annotate_map(true_map, robot_pos, goal_pos)


#         cv2.imshow("Map", true_map)
#         cv2.imshow("Raw", img)                                    # display

#     else:
#         print("Couldn't capture image")
#         break

#     if (cv2.waitKey(1) & 0xFF) == ord('q'):                       # quit when 'Q' key is pressed
#         break

# cap.release()
# cv2.destroyAllWindows()

# Global Path Planning

## Different Approaches
Different approaches exist to find an optimal global path. In the beginning, the use of a visibility graph was considered. To find the shortest path from start to goal, Dijkstra’s algorithm was applied to the visibility graph. The visibility graph is in fact a fast tool to find the optimal path. But it has some incompatibilities with the image processing part and only provides an advantage in terms of computing speed which is not a critical point, as we only search for the global path only once, at the beginning. It's very easy to produce an occupancy grid, it's a simple binarisation of the grayscale map. But finding corners of obstacles is very difficult and restricted to sharp shapes with acute angles. We could use the Harris corner detector which computes the derivative of the image in several direction but this method often doesn't give the expected number of corners by detecting corners in globally straight but locally sawtooth lines. Computing the convex hull of those points is a good way to reduce their number, but it drastically reduces the possible obstacle shapes to sharp and convex shapes. Therefore the visibility graph has many more drawbacks than advantages (especially given the additionnal computation in image processing might compensate the speed of the visibility graph). We prefered to change to another method.
Therefore a cell decomposition/grid based approach was implemented.

### A* on grid array
To find the optimal path from the start to end position, the A* algorithm is used on a binary grid array (occupancy grid). The A* algorithm can be seen as an extension of Dijkstra's algorithm. The grid array represents free spaces with a zero and obstacles (which the robot has to navigate around) with a one. For the implementation, a modified version of the code from the Exercise session 5 of the course MICRO-452 “Basics of Mobile Robotics” was used. The code can be found in the [PathPlanning](src/PathPlanning.py) module.

### A* algorithm theory
The A* algorithm combines a motion cost with a heuristic cost. The motion cost and the heuristic function assign a cost to each cell of the array. The motion cost corresponds to the shortest number of steps from the start position to the given cell (obstacles are avoided). 
The heuristic cost implemented in this case corresponds to the distance between the goal position and the cell (obstacles are not avoided). The final cost for every cell is calculated by adding the corresponding motion and heuristic cost together. To find the optimal path one has to progress step by step to the next adjacent cell with the lowest final cost, starting from the start position until the goal is reached (see fig 5).
Note that the motion and final cost are not calculated for every cell. The algorithm only calculates the cost for “promising” cells (see fig 6).  

> <img src="./imgs/A_star_example.jpg" />

> *Figure 5: This figure shows the motion, heuristic and final cost implemented on an example array. The different costs for every cell are described by the corresponding number in every cell. Note that the red cell is the start, the green cell is the goal, the black cells are obstacles, white cells are free spaces and the gray cells describe the final path.*


> <img src="./imgs/global_path_test.png" />


> *Figure 6: This figure shows the result of the A-star algorithm on a test array. Note, that the white cells are free spaces, the red cells are obstacles, the yellow cells are explored cells and that the optimal path is represented by the blue line.*

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

# Adding the src folder in the current directory as it contains the PathPlanning module
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))
import PathPlanning

## Testing the A* Implementation
Following cell allows the testing of the A* allgorithm on a binary grid array, without needing all the setup (robot and surrounding).

In [ ]:
# max_val_column = 100 # Size of the map
# max_val_row = 50
# occupancy_grid, cmap=PathPlanning.create_random_occupancy_grid(max_val_column, max_val_row)

In [ ]:
# # Define start and end goal -> values have to be changed, that the are on empty spot and that a path between them exists
# test_start=(0,0)    #value has to be changed
# test_goal=(49,99)  #value has to be changed
# path=PathPlanning.generate_global_path(test_start, test_goal, occupancy_grid)
# path=np.fliplr(path)
# print(f"path (form start to goal)")
# print(f"x={path[1]}")
# print(f"y={path[0]}")

# Localistion and Control

## Theory
Once the path planning has been done, we need to get the Thymio robot to follow the planned path.
We do so by implementing a pilot, which uses knowledge of the robot's location relative to its next target to calculate the motor inputs necessary to reach it. 

Unfortunately, the robot does not always have perfect knowledge of its position at all times, so it must be estimated from the limited measurements it does have access to, namely:
- camera localisation
- motor odometry
- ground IR sensors 
- horizontal proximity sensors

Because both the movement process and these measurements are quite noisy or only intermittant, simply dead reckoning the robot's position is not accurate enough. We need to implement a stochasitc state estimator to try to estimate the robot's position from the noisy measurements.

> <img src="./imgs/control-Page-1.png" />

> *Figure 7: Control diagram of the robot and pilot*

### Estimator
A number of stochastic estimation methods exist. The approach chosen in this project, the Kalman Filter, is computationally one of the simplest. It was chosen in the hopes of making the system more reactive. The Kalman filter is normally used for systems with linear behavior, but it can relatively easily be extended to non-linear systems that do not vary too abruptly.

### Control Signal
The Thymio robot may be controlled through its two motors. Our control signal $\mathbf u$ will therefore be : 
$$ \mathbf u = \begin{bmatrix} u_r\\ u_l \end{bmatrix}$$

### State 
The state of the robot $\mathbf x$ is defined as follows :
$$ \mathbf{x} = \begin{bmatrix}x\\ y \\ \phi \\ v_r \\ v_l\end{bmatrix}$$
where $[x , y]^T$ are the robot's position in space, $\phi$ is its angular orientation with respect to the x-axis, and $[v_r , v_l]^T$ are the speed on the right and left wheels, respectively. 

### Motion Model
To apply the extended Kalman Filter, we need to define a motion model $\mathbf x^{+} = g(\mathbf{x,u})$. 
The simplest motion model for a robot such as the Thymio can be calculated in two steps. First, the intermediary values describing the change in angle and in distance are calculated:  
$$ \begin{bmatrix}dl\\ d\phi\end{bmatrix} = B\cdot \mathbf u , \text{ for  } B=c \cdot T_s\begin{pmatrix} 0.5 & 0.5 \\  \frac{1}{d} & -\frac{1}{d} \end{pmatrix} $$ 

where $T_s$ is the sample time and $d$ is the distance between the wheels. $c$ is a conversion constant to convert between Thymio speed and $mm/s$. 
Then the actual states may then be approximated as:
$$\begin{eqnarray}
\phi^{+} &=& \phi + d\phi\\
x^{+}    &=& x + dl \cdot cos(\phi^{+})\\
y^{+}    &=& y + dl \cdot sin(\phi^{+})\\
v_r^{+}  &=& u_r\\
v_l^{+}  &=& u_l
\end{eqnarray}$$
Other motion models were studied, and can be found in the module code.

In order to linearize the motion model for use in the Kalman filter, we must calculate its Jacobian. This can be done analytically, but it is too tedious for this report. The result can be found in the code for this module. 
$$G_{\mathbf{x,u}} = \begin{bmatrix} 
\frac{\partial g}{\partial x}(\mathbf{x,u}) &
\frac{\partial g}{\partial y}(\mathbf{x,u}) & 
\frac{\partial g}{\partial \phi}(\mathbf{x,u}) & 
\frac{\partial g}{\partial v_r}(\mathbf{x,u}) & 
\frac{\partial g}{\partial v_l}(\mathbf{x,u}) 
\end{bmatrix}$$


Finally, we must estimate the uncertainty added by an update step of the motion process. We can measure the uncertainty on the motion process for each motor individually, which gives us $\mathbf{q} = \begin{bmatrix} q_r \\ q_l \end{bmatrix}$ resulting in a coviariance matrix $Q_{\mathbf u}$ with diagonal $\mathbf{q}$.
We can then apply the propagation of covariance formula through the linearized system (the Jacobian $G_{\mathbf{x,u}}$) in order to estimate the uncertainty on the robot state gained due to noise for a single process step:
$$ Q = G_{\mathbf{x,u}} \begin{pmatrix} \mathbf 0 & 0 \\ 0& Q_{\mathbf{u}}\end{pmatrix} G_{\mathbf {x,u}}^T$$
**Note:** This is in fact illegal. The real probablity density of the error added to the robot's state is highly non-linear. It follows a banana-shape, as the angular uncertainty is much larger than the distance uncertainty. However, it was estimated that the robot would have ample chance to localize itself and thus reduce the uncertainty to a level where the non-linearity doesn't matter as much.

### Sensor interpretation 

As mentioned above, the robot has access to several sensors which give it information about the world surrounding it. 
In our application, most of the sensors available to us do not allow a reversible mapping into state-space (i.e: the sensor function $\mathbf y = h(\mathbf x)$ is non-invertible). This is because the map is made up of a repeating grid, so we have no information on the absolute localization (for the IR-sensors). Furthermore, from the point of view of the sensor, there is no way to tell whether the detected line is horizontal or vertical.
We must therefore do a preprocessing step on the raw sensor values to put them into a useable form for our (over)extended Kalman estimator.

A first approach was, whenever a line was detected by a sensor to simply take the coordinate of the closest gridline to the estimated position of that sensor as the "measured value" $\tilde{\mathbf y}$. This however does not take into account the fact that the estimated postion of the robot can be quite uncertain, so the returned "measured values" were very often wrong.

A different approach is to incorporate our estimated uncertainty into the sensor interpretation. Instead of looking for the closest gridline, we look for the most likely coordinates in which we could encounter a gridline, as shown in figure 3 below. These are then taken as $\tilde{\mathbf y}$.
This approach was adapted from the paper : https://www.cs.cmu.edu/~motionplanning/papers/sbp_papers/integrated2/machler_grid_odometry.pdf

> <img src="./imgs/pdf-based_lacation_estimate.png" alt="pdf-based_lacation_estimate" style="width: 200px;"/>

> *Figure 8 : Probability density-based location estimation. The real position of the sensor in space is unkown, so it is modelled as multivariate gaussian probability density function. When a gridline is encountered, we can calculate the position on the gridlines with the highest probablilty, and take this as our new position estimate.*

## Setup

The code for this module can be found under [Robot.py](src/Robot.py), and some generic-use code can also be found in [util.py](src/util.py) and [Field.py](src/Field.py).
The code for creating [threads](src/RepeatedTimer.py) was adapted from code presented in the course MICRO-452 “Basics of Mobile Robotics” 

Below, we define the thread which controls the robot. 

In [ ]:
import os
import sys
import time
import serial
import numpy as np
import matplotlib.pylab as plt
from IPython import display as ds

# Adding the src folder in the current directory as it contains the necessary scripts
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio
import RepeatedTimer
import Field
import Robot
import util

In [ ]:
def robot_thread(field, robot, goals, camera_pos):
    """ do the robot control cycle """

    if not goals :
            raise Exception("No more goals")   #this is bad, but i'm no python expert, so it'll have to do
            return 

    u,local_obstacle = robot.pilot(goals)

    if not local_obstacle :
        robot.do_motion(u)

    sensor_data = robot.do_measure()
    
    if local_obstacle : 
        u = sensor_data[9:11,:]
        u[u>2**15] = u[u>2**15] - 2**16

    y = robot.sensor_interpretation(sensor_data, field)

    if camera_pos.x.size :                                    # if a camera position is available, update
        c0 = np.concatenate((np.eye(3), np.zeros((3,2))), axis = 1)
        y0 = camera_pos.x[0:3,:]
        r0 = np.array([1, 1, .1])**2                      # we assume the camera position is the most precise 
        robot.R = np.diag(np.concatenate((r0, np.diag(robot.R)), axis=0)) # possible to make our lives simpler 
        robot.C = np.concatenate((c0, robot.C))
        y       = np.concatenate((y0, y), axis=0)
        field.xreal  = np.concatenate((field.xreal, camera_pos.x), axis=1)
        camera_pos.x = np.array([])

    xtemp, stemp = robot.kalman_estimator(u, y)

    robot.xodo = robot.motion_model(robot.xodo, u)             # make a purely odometry-based estimate, for comparison

    field.xhat  = np.concatenate((field.xhat, xtemp), axis=1)
    field.xodo  = np.concatenate((field.xodo, robot.xodo), axis=1)
    field.s.append(2*np.sqrt(stemp[0,0]+stemp[1,1]))  

# Local Navigation

The local navigation is programmed with Aseba and thus runs on the Thymio itself. This allows a quick reaction to local obstacles, because the data does not travel to the computer and back.

The goal of the local navigation is to follow alongside the obstacle until the robot is back onto its global path.

To follow the obstacle, mainly the outer infrared (IR) sensors of the Thymio are used, because they are the only sensors that return non-zero values when the Thymio is parallel to the obstacle. Furthermore, only the speed of the wheel which is further away from the obstacle is usually changed. The speed of the wheel which is closer to the obstacle is mostly constant. An exception is made when at least one of the three middle IR sensors returns a non-zero value, which means that the robot is not parallel to the obstacle anymore, but rather perpendicular. In this case the speed of both wheels are reduced, to avoid a collision with the obstacle.

The speed of the wheel which is further away from the obstacle is controlled by a simple proportional controller. This means that the speed of this wheel is proportional to the error (difference between the desired sensor value and the real sensor value). 

In Aseba, a timer emits an event every 200ms. At each of these events, the speed of the wheel closer to the wall is updated, in order to avoid the obstacles. To determine the speeds of the two wheels, first it is determined if the obstacle is located on the left or right side of the robot. Then the speeds are calculated based on the sensor values and the proportional gain. Finally the speed target values of the Thymio are updated with the calculated speeds.

The Python and the Aseba programs have to communicate with each other, to prevent that both programs interfere with each other by giving contradictory commands. For this purpose the <code>event.args</code> variables, which are available for both programs, were used. The variable <code>event.args[0]</code> describes if the robot is in global or local navigation. This is considered by both Python and Aseba programs, such that only one program gives commands to the robot.

To switch from global to local navigation, <code>event.args[0]</code> is changed in Aseba, if a local obstacle is detected with the IR sensors. Furthermore, the flag <code>event.args[0]</code> determines if the Python or the Aseba program controls the robot.

It is a bit harder to switch from global to local navigation. As seen in the course there are multiple solutions to this problem. The chosen solution consists of exiting local navigation if the robot gets back to the global path, as mentioned above. 
Since the global path is described by close points, the robot can be considered to be on the global path, if it is next to such a point. In this case the robot can continue with the global navigation, but it has to ignore the IR sensors for a moment to avoid getting back into local navigation instantly. However, then the possibility exists that the robot collides with the obstacle while ignoring the IR sensors. The solution implemented to solve this issue, is to assure that the end goal and the obstacle are on opposite sides of the robot. This way the robot can exit the local navigation without hitting the obstacle. 

A challenge encountered during the implementation of the local navigation was the robot turning early around the corner and bumping into it with its rear part. This is because the IR sensors are directed forward and that one wheel has a constant speed. The solution implemented to solve this problem is to delay the turning, when the obstacle is not detected by the sensors anymore. This is not an ideal solution but it is functional.

The Aseba code can be found [here](Aseba/local_navigation.aesl), and the Python code is integrated in the [Robot](src/Robot.py) module.

# Final Program

In order to run the final program, all the cells above must be run first. This is to ensure all the necessary functions and libraries have been declared and imported. Furthermore, the Aseba program must be loaded onto the Thymio robot, then the Aseba Studios must be closed.

Once this has been done, the next cell must be run **once**, in order to set up the objects for the next run.

In [ ]:
class Camera_Pos :
    def __init__ (self): 
        pass
camera_pos = Camera_Pos()
robot      = Robot.NewRobot()                                                   # robot 
field      = Field.NewField(1200, 800, 200, 4)                                  # table

robot.th   = Thymio.serial(port="COM15", refreshing_rate=0.1) # set up Thymio communication

cap_size          = (1200, 1000)
global_shape      = (90,  60)
cap               = cv2.VideoCapture(2)                                         # put 1 if an external webcam is used
cap.set(cv2.CAP_PROP_FRAME_WIDTH, cap_size[0])                                  # width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,cap_size[1])                                  # height
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE,1)                                           # set auto exposure
time.sleep(1)                                                                   # give the camera time to adjust

Running the following cell runs a single attempt to reach the goal. 

**Note:** The program opens a window containing the view from the camera. This must be closed in order to proceed.

In [ ]:
#################################################################################
# setup vision
#################################################################################

startpos = np.array([])
end_pos  = np.array([])
while not startpos.size or not end_pos.size:
    success, img  = cap.read()
    if success : 
        success2, true_map  = vis.get_clean_map(img, [field.xmax, field.ymax])
        if success2 :
            startpos  = vis.robot_detection(true_map)                                # try until we get valid positions
            end_pos   = vis.goal_detection(true_map)  
        else:
            print("Could not make map")
    else :
        print("No image captured")
    time.sleep(.1)   
    
true_map = vis.annotate_map(true_map, startpos, end_pos)
cv2.imshow("True Map", true_map)
cv2.waitKey(0)

#################################################################################
# global path finding
#################################################################################

map_global_search = cv2.resize(vis.get_global_search_map(true_map, startpos), global_shape)# apply resizing to reduce computation cost of global search
map_start         = util.rescale_int_coords(startpos, (true_map.shape[1], true_map.shape[0]), global_shape)  # starting point for global search
map_end           = util.rescale_int_coords(end_pos,  (true_map.shape[1], true_map.shape[0]), global_shape)  # end point for global search

occupancy_grid    = np.flipud(np.asarray(map_global_search))                     # make occupancy grid and change coordinate system           
path              = PathPlanning.generate_global_path((map_start[1], map_start[0]), (map_end[1], map_end[0]), occupancy_grid)
path              = util.adapt_pathfinder_coords(path, field, occupancy_grid.shape)

if not path.size :
    raise Exception("No Path Found")

#################################################################################
# setup robot 
#################################################################################

Tr = .2                                                                          # robot update timestep
Tc = 2                                                                           # camera update timestep

robot.xhat  = startpos.copy()                                                    # setup robot
robot.xodo  = startpos.copy()
robot.s     = np.diag([1, 1, .1, 20, 20])**2

robot.d     = 90
robot.set_Ts(Tr)
robot.maxv  = 10
robot.maxw  = np.pi/16

field.goals = path
field.xreal = startpos.copy()                                                    # prepare record of (mis)deeds
field.xhat  = robot.xhat.copy()
field.xodo  = robot.xodo.copy()
field.s     = [2*np.sqrt(robot.s[0,0]+robot.s[1,1])]

goals       = np.ndarray.tolist(field.goals)

camera_pos.x = np.array([])                                                      # communication variable


#################################################################################
# run
#################################################################################
T = 120                                                                           # maximum run time allowed

rt = RepeatedTimer.Thread(Tr, robot_thread,  field, robot, goals, camera_pos);   # start threads
ct = RepeatedTimer.Thread(Tc, camera_thread, cap, camera_pos);

try:
    for i in range(T):
        time.sleep(1) 
        plt.clf();
        plt.gcf().set_size_inches(12,8)
        plt.imshow(occupancy_grid,cmap=colors.ListedColormap(['white', 'black']), origin = 'lower', extent=(field.xmin, field.xmax, field.ymin, field.ymax))
        field.plot()
        ds.display(plt.gcf())
        ds.clear_output(wait=True)
        
        if not goals : 
            print("arrived")
            break

finally:
    rt.stop()
    ct.stop()
    robot.th.set_var("event.args", 2)
    robot.do_motion(np.zeros((2,1)))

Run this cell to stop the robot and close the serial connection:

In [ ]:
robot.do_motion(np.zeros((2,1)))
robot.th.close()

# Results

## Normal Run
If the lighting conditions are good, we can generally achieve quite good results in a normal run, due to constant updates from the camera and continuous movement of the robot. 

A typical such run can be seen below. We plot the estimated position (red) against the position given by the camera (green) and the odometry only-based estimation (yellow). One can see that the odometry only-based estimation has a relatively large drift. 

In [1]:
%%HTML
<div align="middle">
<video width="80%" controls>
    <source src="./videos/run_2.mp4">
</video></div> <br>
<i>Video 1: This video shows a successful test: The robot follows the planned path, while avoiding the local (white block) and global obstacles (black objects).</i>

## No Vision
Since our system relies quite heaviliy on vision, we tested it when the vision is blocked. 
The vision can easily be blocked by covering one of the blue corner markers. In that case, no position is returned by the vision module. 

The result, while not perfect is acceptable. We plot the estimated position (red) against the position given by the camera (green) and the odometry only-based estimation (yellow). One can see that the odometry only-based estimation has a relatively large drift. The dotted blue lines represent the uncertainty of the robot as to its estimation, which is too optimistic, as the real position is far outside the 90% certainty zone. 

In [2]:
%%HTML
<div align="middle">
<video width="80%" controls>
    <source src="./videos/no_camera.mp4">
</video></div> <br>
<i>Video 2: This video shows the robot successfully finding it’s path to the goal position, without using the camera (one of the blue sports is covered with a black sheet of paper during the global navigation)</i>

## Kidnapping
Since the localisation algorithm is a Kalman filter, it does not deal well with non-linearites and sudden jumps. 
One way to demonstrate this is to put the robot in a kidnapping scenario. 

The robot eventually recovers, but it takes some time.


In [3]:
%%HTML
<div align="middle">
<video width="80%" controls>
    <source src="./videos/kidnapping.mp4">
</video></div>
<i>Video 3: This video shows the robot successfully dealing with a kidnapping situation. Also, the goal dot came unstuck.</i>

# Conclusion

## Image Processing
To summarise, image processing provides quite good results in regards with the methods tested. There are still failures in localising the robot : in hindsight, the red color wasn't such a good choice due to the red LEDs on the robot. There are also some uncertainties due the light (intensity and color) and shadows. In general some robustness steps have been implemented, but the speed of the algorithm could be much better. We aimed for a robust rather than a fast method.

## Global Path Planning

The global path finding part returns successively the shortest path from the start to goal. But the implemented A* algorithm uses a not negligible amount of time and storage space to fulfill this task. Nevertheless the A* algorithm seems to be sufficient for our task, because it is only executed once in the beginning of the experiment.

## Localisation and Control
The employed localisation method, a Kalman estimator on the position of the robot does not work very well. The non-linearities of the system make it such that a gaussian probability density function does not correctly represent the uncertainty.

Possible improvements which were considered, but not investigated due to lack of time, are:
- using a Kalman filter in the $[dl, d\phi]^{T}$ space, where the robot's motion model is linear
- using a modified filter and multiple gaussians to better represent the true probablilty distribution
- using a particle filter (this would make the system very slow though)

However, thanks to the use of frequent position updated from the camera, we were able to keep the uncertainties low and thus the probabiltiy density function *nearly* gaussian, and thus achieve an acceptable overall result. 

## Local Navigation

To get the local navigation to work, the parameters had to be tweaked which lead to extensive testing. The hand-off between Python and Aseba did not work well at first. However, after letting the robot get out of the local obstacle avoidance mode only if the obstacle was not between the robot and the goal, the switching between local and global navigation was successful. Differences in lighting conditions for different group members meant that the robot would sometimes get too close to the obstacle. Thus the robot would bump into the obstacle. The solution of this issue was to adjust the parameters. 

Due to time constraints, avoiding global obstacles during the local navigation was not implemented. However, if the local obstacle is not placed next to a global obstacle, this is not a problem and the local navigation works fine.

## Overall

Due to the current COVID-19 pandemic and the geographical dipersion of the group members, all collaboration was done online. 
This unfortunately made combining the modules, testing and debugging them extremely time-consuming, especially since only one "environment" was available.

We are therefore quite satisfied with the results of this project, though of course, had we had the time, we would have liked to improve on a number of points.